# Asynchronous Programming

All network I/O is done with coroutines in `asyncio`, but not file I/O. However, file I/O is also "blocking"——in the sense that reading/writing files takes thousands of times longer than reading/writing to RAM. If you're using Network-Attached Storage, it may even involve network I/O under te covers. 

Since Python 3.9, the `asyncio.to_thread` coroutine makes it easy tot delegate file I/O to a thread pool provided by `asyncio`. 

### An asyncio Example: Probing Domains

[blogdom.py](./blogdom.py)

## Throttling Requests with a Semaphore

An `asyncio.Semaphore` has an internal counter that is decremented whenever we `await` on the `.acquire()` coroutine method, and incremented when we call the `.release()` method——which is not coroutine because it never blocks. 

Awating on `.acquire()` cause no delay when the counter is greater than zero, but if the counter is zero, `.acquire()` suspends the awaiting coroutine until some other coroutine calls `.release()` on the same `Semaphore`. 

**Example code:** [flags2_asyncio.py: 183](./flags_asyncio/flags2_asyncio.py:183)

## Writing asyncio Servers

### A FastAPI Web Service

[FastAPI: web_mojifinder](./web_mojifinder.py)

### An asyncio TCP Server

[TCP Server: tcp_mojifinder](./tcp_mojifinder.py)

## Python's async console

run this command line: 

```shell
python -m asyncio
```

## Asynchronous Context Managers

Sample code from the documentation of the asyncpg PostgreSQL driver. 

```python
tr = connection.transaction()
await tr.start()

try:
    await connection.execute("INSERT INTO mytable VALUES (1, 2, 3)")
except:
    await tr.rollback()
    raise
else:
    await tr.commit()
```

Here is an **example** that illustrates the use of `async for` to iterate over the rows of a database cursor:

```python
async def go():
    pool = await aiopg.create_pool(dsn)
    async with conn.cursor() as cur:
        await cur.execute("SELECT 1")
        ret = []
        async for row in cur:
            ret.append(row)
        assert ret == [(1,)]
```


### Asynchronous generators as context managers

```python
import asyncio
from contextlib import asynccontextmanager

@asynccontextmanager
async def web_page(url):
    loop = asyncio.get_running_loop()
    data = await loop.run_in_executor(
        None, download_webpage, url)
    # Lines before this `yield` expression will become the `__aenter__` coroutine method. 
    yield data
    # Lines after this `yield` expression will become the `__aexit__` coroutine method. 
    await loop.run_in_executor(None, update_stats, url)

async with web_page('google.com') as data:
    process(data)
```

#### Asynchronous generators VS Native generators

- An asynchronous generator always has a `yield` expression in its body, while a native coroutine never contains `yield`. 
- An asynchronous generator can only use empty `return` statements, while a native coroutine may `return` some value than `None`. 
- Native coroutines are awaitable. Asynchronous generators are not awaitable. 

## async Beyond asyncio: `curio`

In [2]:
from curio import run, TaskGroup
import curio.socket as socket
from keyword import kwlist

MAX_KEYWORD_LEN = 4


# `probe` doesn't need to get the event loop. 
async def probe(domain: str) -> tuple[str, bool]: 
    try:
        await socket.getaddrinfo(domain, None)  
    except socket.gaierror:
        return (domain, False)
    return (domain, True)


async def main() -> None:
    names = (kw for kw in kwlist if len(kw) <= MAX_KEYWORD_LEN)
    domains = (f'{name}.dev'.lower() for name in names)
    # A `TaskGroup` is a core concept in `Curio`, to monitor and control several coroutines,
    # and to make sure they are all executed and cleaned up. 
    async with TaskGroup() as group:  
        for domain in domains:
            # `TaskGroup.spawn` is how you start a coroutine. 
            await group.spawn(probe, domain)  
        # Iterating with `async for` over a `TaskGroup` yields `Task` instances as each is completed. 
        # This correspons to the previous examples using `for ... as_completed(...)`
        async for task in group:  
            domain, found = task.result
            mark = '+' if found else ' '
            print(f'{mark} {domain}')


In [3]:
run(main())

+ def.dev
+ try.dev
+ in.dev
+ as.dev
+ and.dev
  or.dev
  for.dev
  none.dev
  elif.dev
  with.dev
  if.dev
  else.dev
+ from.dev
  is.dev
+ not.dev
  true.dev
  pass.dev
+ del.dev
